In [17]:
# ENTER YOUR NAME AND ID HERE
# Yonghwan Kim A11746276

# Project-2: Locality Sensitive Hashing

In [18]:
import random
import os
import numpy as np
import re

from random import randrange
from shutil import copyfile
from collections import defaultdict

Suspicious datasets for this project will be in <b>Suspicious</b> directory


Your query datasets will be in <b>Original</b> directory


You have to use any one original Wikipedia article from <b>Original</b> for <b>Fact Checks</b> steps

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Original Directory
originalAll = '/content/drive/My Drive/Project2/Original'
# Suspicious Directory
suspicious = '/content/drive/My Drive/Project2/Suspicious'

### STEP - 1: Shingling (20 points)

In [21]:
# Type your code here... 
# Create necessary number of cells below this cell
k = 3
# NOTE: No complex text processing is required
# convert just upper case characters to lower case

# sort suspicious file names
susFiles = sorted(os.listdir(suspicious))
# sort original file names
orgFiles = sorted(os.listdir(originalAll))

## shigling seperated by file
def shinglingFunc(gram, dir):
    # retrieve file names from directory
    filelist = os.listdir(dir)
    # A list for shingles
    shingleList = []
    # Open files
    for f in sorted(filelist):
        with open(os.path.join(dir, f), "r", encoding = "ISO-8859-1") as fileOrg: # open a file with encoding ISO-8859-1 to recover broken Unicode.
            i = 0
            all = ""
            fileShingle = [] # Shingles from a document
            for line in fileOrg: # A line from a document
                if not line is "\n":
                    all = all + str(line)
            all = all.lower().replace("\n", " ").replace("\t", "") # change all to lower case and remove next lines and tabs
            all = re.sub('[^a-z0-9 ]+', '', all) # remove special characters to increase matching rate later
            singleList = all.split(" ") # extract words from a line
            singleList = [x.strip(' ') for x in singleList]
            singleList = list(filter(None, singleList)) # remove shingles contain nothing
            gard = True # gard value to escape loop
            index = 0
            counter = 0
            while gard:
                # end of the list, escape loop
                if index == len(singleList)-gram:
                    gard = False
                counter = index
                shingle = []
                for i in range(gram):
                    shingle.append(singleList[counter])
                    counter = counter + 1
                fileShingle.append(tuple(shingle))
                index = index + 1
            shingleList.append(set(fileShingle)) # change list to set to remove duplicates
    return shingleList

In [22]:
# function to combine all shingles from each documents
def findUniqueShingle(shingles):
    uniqueSets = set()
    for u in shingles:
        uniqueSets = uniqueSets.union(u)
    return uniqueSets
# print the number of unique shingles
def printShingles(uniqueShingleList):
    print(len(uniqueShingleList))

In [23]:
# 3 grams, k = 3
threegram = shinglingFunc(k, suspicious)
uniqueThreegram = findUniqueShingle(threegram)
print("k=3")
printShingles(uniqueThreegram)
# 4 grams, k = 4
fourgram = shinglingFunc(k+1, suspicious)
uniqueFourgram = findUniqueShingle(fourgram)
print("k=4")
printShingles(uniqueFourgram)
# 5 grams, k = 5
fivegram = shinglingFunc(k+2, suspicious)
uniqueFivegram = findUniqueShingle(fivegram)
print("k=5")
printShingles(uniqueFivegram)

k=3
12866
k=4
14383
k=5
15071


In [24]:
# Type your code to get the 5-shingle index here
# function indexing 5 grams shingle list
def creatIndex(shigleList):
    fiveGdict = dict()
    index = 0
    for shingle in shigleList:
        fiveGdict[index] = shingle
        index = index + 1
    return fiveGdict

fivegramIndex = creatIndex(uniqueFivegram)

### STEP - 2: Min-Hashing (40 points)

In [25]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions

# test
# hash_functions = get_hash_functions(5000,50)
# print(hash_functions)

In [26]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5
L = 50

total = len(uniqueFivegram)
hashOf50 = get_hash_functions(total, L) # 50 permutations
hashOf100 = get_hash_functions(total, L + 50) # 100 permutations
hashOf200 = get_hash_functions(total, L + 150) # 200 permutations
hashOf500 = get_hash_functions(total, L + 450) # 500 permutations
hashOf1000 = get_hash_functions(total, L + 950) # 1000 permutations
hashs = [hashOf50, hashOf100, hashOf200, hashOf500, hashOf1000] # list of each permutations

In [27]:
# Type your code here to get the final signature matrix S here

# signature matrix generater taking permutations, 5 shingle indexes and 5 shingle list for each document
def signatureMatrix(hash, fivegramIndex, documents):
    S = np.full(( len(hash), len(documents) ), np.inf) # create a matrix containing infinite, ex) row (hash) 50, column (documents) 95
    column = 0
    for doc in documents:
        inputMat = [0]*len(fivegramIndex) # initialize one input matrix of one document, row (shingles), column (documents)
        for k, v in fivegramIndex.items():
            sv = {v}
            if sv.issubset(doc): # if match save 1
                inputMat[k] = 1
        # with one input matrix, create a signature matrix for the document
        for j in range(len(inputMat)):
            if inputMat[j] == 1:
                row = 0
                for h in hash:
                    if h[j] < S[row][column]:
                        S[row][column] = h[j]
                    row = row + 1
        column = column + 1
    return S

In [28]:
# Type your code here to do the fact check 
#      with any one query document in the 'Original' directory

original = 'Original/orig_taskc.txt'

# STEP-1: Generate 5-shingles 
    # (if any shingles are not present in your shingle index, simply ignore them)

# Generate 5-shingles for Original files
originalFivegram = shinglingFunc(k+2, originalAll)
uniqueOriginalFivegram = findUniqueShingle(originalFivegram)
    
# STEP-2: Generate signature vector from L hash functions

# Suspicious signature matrix
sigMatrices = []

for permut in hashs:
    sigMatrices.append(signatureMatrix(permut, fivegramIndex, fivegram))

# Original signature matrix
sigMatricesOrg = []

for permut in hashs:
    sigMatricesOrg.append(signatureMatrix(permut, fivegramIndex, originalFivegram))

# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
    # and all other documents 

# Threshold value
t = 0.85

# function to calculate jaccard similarity
def jaccardSim(susSig, origSig, ind, t):
    simCount_x = 0
    simCount_y = 0
    for c in range(len(origSig)):
        if susSig[c] == origSig[c]:
            simCount_x = simCount_x + 1
        elif susSig[c] != origSig[c]:
            simCount_y = simCount_y + 1
    sim = simCount_x/(simCount_x+simCount_y)
    if sim > t: # return similiarty value over the threshold point
        return [sim, ind]
    else:
        return []
        
# STEP-4: Facts Checks
# For each L = {50,100,200,500,1000}, report all documents (file_names) below that have Jaccard similarity > t=0.85
# Sort the documents in decreasing order of the Jaccard similarity

# find documents that has simiiliarty over 0.85
for c in range(len(sigMatricesOrg)):
    print("Original file: ", orgFiles[c]) # name of the original file
    for i, h in enumerate(hashs):
        print("Hash: ", len(h)) # Hash L's length
        matchDoc = []
        for ind in range(len(sigMatrices[i][i])):
            match = jaccardSim(sigMatrices[i][:, ind], sigMatricesOrg[i][:, c], susFiles[ind], t)
            if match:
                matchDoc.append(match)
        matchDoc = sorted(matchDoc, reverse=True) # sort by similarity value in decreasing order
        for md in matchDoc:
           print("Suspicious file: ", md[1], " similarity:", md[0]) # name of the suspicious file
    print()

Original file:  orig_taska.txt
Hash:  50
Suspicious file:  g4pC_taska.txt  similarity: 0.88
Hash:  100
Suspicious file:  g0pE_taska.txt  similarity: 0.93
Suspicious file:  g4pC_taska.txt  similarity: 0.88
Hash:  200
Suspicious file:  g0pE_taska.txt  similarity: 0.865
Hash:  500
Suspicious file:  g0pE_taska.txt  similarity: 0.888
Hash:  1000
Suspicious file:  g0pE_taska.txt  similarity: 0.885
Suspicious file:  g4pC_taska.txt  similarity: 0.857

Original file:  orig_taskb.txt
Hash:  50
Hash:  100
Hash:  200
Hash:  500
Hash:  1000

Original file:  orig_taskc.txt
Hash:  50
Hash:  100
Hash:  200
Hash:  500
Hash:  1000

Original file:  orig_taskd.txt
Hash:  50
Suspicious file:  g3pA_taskd.txt  similarity: 0.94
Hash:  100
Suspicious file:  g3pA_taskd.txt  similarity: 0.95
Hash:  200
Suspicious file:  g3pA_taskd.txt  similarity: 0.95
Hash:  500
Suspicious file:  g3pA_taskd.txt  similarity: 0.966
Hash:  1000
Suspicious file:  g3pA_taskd.txt  similarity: 0.963

Original file:  orig_taske.txt
Has

### STEP - 3: LSH (30 points)

In [29]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

b = 50 # bands
r = 20 # rows per band
B = 199 # buckets

# Suspicious signature with only L = 1000
susSigL1000 = sigMatrices[4]
# Original signature with only L = 1000
orgSigL1000 = sigMatricesOrg[4]

# make a random list for hashing
def randomRowList(r):
    randomlist = random.sample(range(1,r+1), r)
    return randomlist

# seperate a matrix into n band size
def slicingToBand(matrix, bandSize):
    bands = np.array_split(matrix, bandSize)
    return bands

# hashing into each bucket
def bucketHashing(bands, B, randList, index):
    hashbuckets = defaultdict(set) # a dictionary with sets inside

    for i,band in enumerate(bands):
        bandInd = tuple(list(band[:,index])) # one band
        col = []
        for x, row in enumerate(bandInd): # rows of a band
            col.append(row * randList[x]) # each row * ai
        sigma = sum(col) # add every data
        h = sigma % B # sigma mod 199
        hashbuckets[int(h)].add(i) # add to proper bucket
    return hashbuckets

In [30]:
# Type your code here to do generate candidate documents
# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

# STEP - 1: Split each original document signature vector into b bands of r rows

# generate random list for hasing
rlist = randomRowList(r)
# generate bands for original signature
orgBands = slicingToBand(orgSigL1000, b)
# generate bands for suspicious signature
susBands = slicingToBand(susSigL1000, b)

# initialize candidate pair in dictionary list
candidatePair = defaultdict(list)

for orgdoc in range(len(orgSigL1000[0])):
    orgBucket = bucketHashing(orgBands, B, rlist, orgdoc) # an orignial bucket
    for susdoc in range(len(susSigL1000[0])):
        susBucket = bucketHashing(susBands, B, rlist, susdoc) # a suspicious bucket
        for bi in range(B):
            if len(orgBucket[bi]) != 0 and len(susBucket[bi]) != 0: # both side is not empty
                if orgBucket[bi].intersection(susBucket[bi]): # if they have (a) same band numbers
                    candidatePair[orgdoc].append(susdoc) # add to candidate pair list
                    break # found one pair, no need to search further

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)

# create candidate signature matrices with same hash function of L=1000
candidSigMatrices = []
candidSigMatrices.append(signatureMatrix(hashs[4], fivegramIndex, fivegram))

candidSigMatricesOrg = []
candidSigMatricesOrg.append(signatureMatrix(hashs[4], fivegramIndex, originalFivegram))

In [31]:
# Type your code here to do the fact check
# Calculate Jaccard similarity of the oiginal document with only 
    # candidate documents

# jaccard similarity with only cadidate documents
for c in range(len(candidatePair)):
    print("-----------------------------------------------------------")
    print("Original file: ", orgFiles[c])
    matchDoc = []
    for ind in candidatePair[c]:
        match = jaccardSim(candidSigMatrices[0][:, ind], candidSigMatricesOrg[0][:, c], susFiles[ind], t)
        if match:
            matchDoc.append(match)
    matchDoc = sorted(matchDoc, reverse=True) # sort in decreasing order
    for md in matchDoc:
        print("Suspicious file: ", md[1], " similarity:", md[0])      
    print("-----------------------------------------------------------")

-----------------------------------------------------------
Original file:  orig_taska.txt
Suspicious file:  g0pE_taska.txt  similarity: 0.885
Suspicious file:  g4pC_taska.txt  similarity: 0.857
-----------------------------------------------------------
-----------------------------------------------------------
Original file:  orig_taskb.txt
-----------------------------------------------------------
-----------------------------------------------------------
Original file:  orig_taskc.txt
-----------------------------------------------------------
-----------------------------------------------------------
Original file:  orig_taskd.txt
Suspicious file:  g3pA_taskd.txt  similarity: 0.963
-----------------------------------------------------------
-----------------------------------------------------------
Original file:  orig_taske.txt
-----------------------------------------------------------


In [36]:
# function to find false positives
def falsePositive(susSig, origSig, ind, t):
    simCount_x = 0
    simCount_y = 0
    for c in range(len(origSig)):
        if susSig[c] == origSig[c]:
            simCount_x = simCount_x + 1
        elif susSig[c] != origSig[c]:
            simCount_y = simCount_y + 1
    sim = simCount_x/(simCount_x+simCount_y)
    if sim <= t: # if candidate similarity is lesser or equal to 0.85, then it is fale positive
        return [sim, ind, "False Positive"]

numOfFalsePositive = 0  
for c in range(len(candidatePair)):
    print("-----------------------------------------------------------")
    print("Original file: ", orgFiles[c])
    matchDoc = []
    for ind in candidatePair[c]: # candidate documents
        match = falsePositive(candidSigMatrices[0][:, ind], candidSigMatricesOrg[0][:, c], susFiles[ind], t)
        if match:
            matchDoc.append(match)
    matchDoc = sorted(matchDoc, reverse=True)
    numOfFalsePositive = numOfFalsePositive + len(matchDoc)
    for md in matchDoc:
        print("Suspicious file: ", md[1], " similarity:", md[0], " ", md[2])
    print("-----------------------------------------------------------")
print("Total Number of False Positives: ", numOfFalsePositive)

-----------------------------------------------------------
Original file:  orig_taska.txt
Suspicious file:  g1pD_taska.txt  similarity: 0.05   False Positive
Suspicious file:  g2pC_taskd.txt  similarity: 0.013   False Positive
Suspicious file:  g0pB_taskd.txt  similarity: 0.013   False Positive
Suspicious file:  g3pA_taska.txt  similarity: 0.012   False Positive
Suspicious file:  g4pD_taske.txt  similarity: 0.011   False Positive
Suspicious file:  g4pC_taskd.txt  similarity: 0.009   False Positive
Suspicious file:  g4pB_taskc.txt  similarity: 0.009   False Positive
Suspicious file:  g2pB_taskd.txt  similarity: 0.009   False Positive
Suspicious file:  g2pA_taskc.txt  similarity: 0.008   False Positive
Suspicious file:  g1pA_taska.txt  similarity: 0.008   False Positive
Suspicious file:  g0pD_taskb.txt  similarity: 0.008   False Positive
Suspicious file:  g4pD_taskc.txt  similarity: 0.007   False Positive
Suspicious file:  g1pA_taskb.txt  similarity: 0.007   False Positive
Suspicious fi

In [37]:
# function to find false negatives (same as jaccard similarity function)
def falseNegative(susSig, origSig, ind, t):
    simCount_x = 0
    simCount_y = 0
    for c in range(len(origSig)):
        if susSig[c] == origSig[c]:
            simCount_x = simCount_x + 1
        elif susSig[c] != origSig[c]:
            simCount_y = simCount_y + 1
    sim = simCount_x/(simCount_x+simCount_y)
    if sim > t:
        return [sim, ind, "False Negative"]

# find non candidate pairs to find false negatives
noncandidatePair = defaultdict(list)
for c in candidatePair:
    for rn in range(len(susFiles)):
        if rn not in candidatePair[c]:
            noncandidatePair[c].append(rn)
numOfFalseNegatives = 0
# run a test with non candidate pairs which has higher similarty over 0.85
for c in range(len(noncandidatePair)):
    print("---------------------------------------------------------------------")
    print("Original file: ", orgFiles[c])
    matchDoc = []
    for ind in noncandidatePair[c]:
        match = falseNegative(candidSigMatrices[0][:, ind], candidSigMatricesOrg[0][:, c], susFiles[ind], t)
        if match:
            matchDoc.append(match)
    matchDoc = sorted(matchDoc, reverse=True)
    numOfFalseNegatives = numOfFalseNegatives + len(matchDoc)
    for md in matchDoc:
        print("Suspicious file: ", md[1], " similarity:", md[0], " ", md[2])      
    print("---------------------------------------------------------------------")
print("Total Number of False Negatives: ", numOfFalseNegatives)

---------------------------------------------------------------------
Original file:  orig_taska.txt
---------------------------------------------------------------------
---------------------------------------------------------------------
Original file:  orig_taskb.txt
---------------------------------------------------------------------
---------------------------------------------------------------------
Original file:  orig_taskc.txt
---------------------------------------------------------------------
---------------------------------------------------------------------
Original file:  orig_taskd.txt
---------------------------------------------------------------------
---------------------------------------------------------------------
Original file:  orig_taske.txt
---------------------------------------------------------------------
Total Number of False Negatives:  0


In [34]:
# sorted in decreasing order by similarity > t=0.85 based on each original files
# -----------------------------------------------------------
# Original file:  orig_taska.txt
# Suspicious file:  g0pE_taska.txt  similarity: 0.917
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskb.txt
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskc.txt
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskd.txt
# Suspicious file:  g3pA_taskd.txt  similarity: 0.959
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taske.txt
# -----------------------------------------------------------

In [35]:
# False positives: 104
# -----------------------------------------------------------
# Original file:  orig_taska.txt
# Suspicious file:  g1pD_taska.txt  similarity: 0.05   False Positive
# Suspicious file:  g2pC_taskd.txt  similarity: 0.013   False Positive
# Suspicious file:  g0pB_taskd.txt  similarity: 0.013   False Positive
# Suspicious file:  g3pA_taska.txt  similarity: 0.012   False Positive
# Suspicious file:  g4pD_taske.txt  similarity: 0.011   False Positive
# Suspicious file:  g4pC_taskd.txt  similarity: 0.009   False Positive
# Suspicious file:  g4pB_taskc.txt  similarity: 0.009   False Positive
# Suspicious file:  g2pB_taskd.txt  similarity: 0.009   False Positive
# Suspicious file:  g2pA_taskc.txt  similarity: 0.008   False Positive
# Suspicious file:  g1pA_taska.txt  similarity: 0.008   False Positive
# Suspicious file:  g0pD_taskb.txt  similarity: 0.008   False Positive
# Suspicious file:  g4pD_taskc.txt  similarity: 0.007   False Positive
# Suspicious file:  g1pA_taskb.txt  similarity: 0.007   False Positive
# Suspicious file:  g0pC_taskb.txt  similarity: 0.007   False Positive
# Suspicious file:  g0pB_taske.txt  similarity: 0.006   False Positive
# Suspicious file:  g2pA_taskd.txt  similarity: 0.005   False Positive
# Suspicious file:  g2pC_taskb.txt  similarity: 0.004   False Positive
# Suspicious file:  g1pB_taska.txt  similarity: 0.003   False Positive
# Suspicious file:  g0pA_taskb.txt  similarity: 0.002   False Positive
# Suspicious file:  g2pE_taske.txt  similarity: 0.001   False Positive
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskb.txt
# Suspicious file:  g2pE_taskb.txt  similarity: 0.082   False Positive
# Suspicious file:  g4pD_taskb.txt  similarity: 0.014   False Positive
# Suspicious file:  g0pA_taska.txt  similarity: 0.014   False Positive
# Suspicious file:  g2pA_taske.txt  similarity: 0.011   False Positive
# Suspicious file:  g2pB_taske.txt  similarity: 0.008   False Positive
# Suspicious file:  g2pB_taskb.txt  similarity: 0.008   False Positive
# Suspicious file:  g4pD_taskc.txt  similarity: 0.007   False Positive
# Suspicious file:  g4pD_taska.txt  similarity: 0.007   False Positive
# Suspicious file:  g3pC_taskb.txt  similarity: 0.007   False Positive
# Suspicious file:  g4pB_taske.txt  similarity: 0.006   False Positive
# Suspicious file:  g1pA_taskc.txt  similarity: 0.006   False Positive
# Suspicious file:  g0pB_taske.txt  similarity: 0.006   False Positive
# Suspicious file:  g3pA_taskc.txt  similarity: 0.005   False Positive
# Suspicious file:  g0pC_taskd.txt  similarity: 0.005   False Positive
# Suspicious file:  g2pE_taskc.txt  similarity: 0.004   False Positive
# Suspicious file:  g1pD_taskc.txt  similarity: 0.004   False Positive
# Suspicious file:  g0pA_taskc.txt  similarity: 0.004   False Positive
# Suspicious file:  g4pE_taskd.txt  similarity: 0.003   False Positive
# Suspicious file:  g3pA_taska.txt  similarity: 0.003   False Positive
# Suspicious file:  g2pA_taskd.txt  similarity: 0.003   False Positive
# Suspicious file:  g0pB_taskc.txt  similarity: 0.003   False Positive
# Suspicious file:  g4pE_taskc.txt  similarity: 0.002   False Positive
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskc.txt
# Suspicious file:  g0pB_taskc.txt  similarity: 0.516   False Positive
# Suspicious file:  g2pA_taskc.txt  similarity: 0.315   False Positive
# Suspicious file:  g1pB_taskc.txt  similarity: 0.067   False Positive
# Suspicious file:  g4pD_taskd.txt  similarity: 0.016   False Positive
# Suspicious file:  g0pB_taskd.txt  similarity: 0.015   False Positive
# Suspicious file:  g2pA_taska.txt  similarity: 0.014   False Positive
# Suspicious file:  g1pA_taska.txt  similarity: 0.008   False Positive
# Suspicious file:  g4pC_taske.txt  similarity: 0.007   False Positive
# Suspicious file:  g2pA_taskb.txt  similarity: 0.007   False Positive
# Suspicious file:  g1pD_taska.txt  similarity: 0.006   False Positive
# Suspicious file:  g4pD_taskb.txt  similarity: 0.005   False Positive
# Suspicious file:  g4pB_taskd.txt  similarity: 0.005   False Positive
# Suspicious file:  g3pB_taske.txt  similarity: 0.005   False Positive
# Suspicious file:  g3pB_taskb.txt  similarity: 0.005   False Positive
# Suspicious file:  g3pB_taska.txt  similarity: 0.005   False Positive
# Suspicious file:  g2pA_taskd.txt  similarity: 0.005   False Positive
# Suspicious file:  g0pD_taskb.txt  similarity: 0.005   False Positive
# Suspicious file:  g2pB_taskd.txt  similarity: 0.004   False Positive
# Suspicious file:  g2pB_taska.txt  similarity: 0.004   False Positive
# Suspicious file:  g1pD_taskc.txt  similarity: 0.004   False Positive
# Suspicious file:  g0pD_taska.txt  similarity: 0.004   False Positive
# Suspicious file:  g0pA_taske.txt  similarity: 0.004   False Positive
# Suspicious file:  g3pC_taskc.txt  similarity: 0.003   False Positive
# Suspicious file:  g3pB_taskd.txt  similarity: 0.003   False Positive
# Suspicious file:  g3pC_taske.txt  similarity: 0.002   False Positive
# Suspicious file:  g2pC_taskd.txt  similarity: 0.002   False Positive
# Suspicious file:  g1pB_taska.txt  similarity: 0.002   False Positive
# Suspicious file:  g4pD_taska.txt  similarity: 0.001   False Positive
# Suspicious file:  g4pB_taska.txt  similarity: 0.001   False Positive
# Suspicious file:  g3pC_taskb.txt  similarity: 0.001   False Positive
# Suspicious file:  g2pE_taska.txt  similarity: 0.001   False Positive
# Suspicious file:  g0pE_taskd.txt  similarity: 0.001   False Positive
# Suspicious file:  g0pC_taskd.txt  similarity: 0.001   False Positive
# Suspicious file:  g0pA_taskb.txt  similarity: 0.0   False Positive
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taskd.txt
# Suspicious file:  g2pC_taskd.txt  similarity: 0.101   False Positive
# Suspicious file:  g1pB_taskd.txt  similarity: 0.052   False Positive
# Suspicious file:  g3pC_taskd.txt  similarity: 0.024   False Positive
# Suspicious file:  g0pC_taska.txt  similarity: 0.011   False Positive
# Suspicious file:  g2pA_taskb.txt  similarity: 0.01   False Positive
# Suspicious file:  g4pD_taskb.txt  similarity: 0.008   False Positive
# Suspicious file:  g4pB_taskb.txt  similarity: 0.008   False Positive
# Suspicious file:  g0pE_taska.txt  similarity: 0.008   False Positive
# Suspicious file:  g2pB_taskc.txt  similarity: 0.007   False Positive
# Suspicious file:  g2pB_taskb.txt  similarity: 0.007   False Positive
# Suspicious file:  g4pD_taskc.txt  similarity: 0.006   False Positive
# Suspicious file:  g3pA_taskb.txt  similarity: 0.006   False Positive
# Suspicious file:  g0pE_taske.txt  similarity: 0.004   False Positive
# Suspicious file:  g4pC_taske.txt  similarity: 0.003   False Positive
# Suspicious file:  g2pE_taskc.txt  similarity: 0.003   False Positive
# Suspicious file:  g2pE_taskb.txt  similarity: 0.003   False Positive
# Suspicious file:  g4pD_taska.txt  similarity: 0.002   False Positive
# Suspicious file:  g2pC_taskc.txt  similarity: 0.002   False Positive
# -----------------------------------------------------------
# -----------------------------------------------------------
# Original file:  orig_taske.txt
# Suspicious file:  g1pB_taske.txt  similarity: 0.215   False Positive
# Suspicious file:  g0pE_taske.txt  similarity: 0.192   False Positive
# Suspicious file:  g4pD_taske.txt  similarity: 0.109   False Positive
# Suspicious file:  g4pB_taska.txt  similarity: 0.015   False Positive
# Suspicious file:  g0pC_taskc.txt  similarity: 0.013   False Positive
# Suspicious file:  g2pE_taske.txt  similarity: 0.011   False Positive
# Suspicious file:  g1pB_taska.txt  similarity: 0.01   False Positive
# Suspicious file:  g4pE_taskd.txt  similarity: 0.007   False Positive
# Suspicious file:  g4pD_taskd.txt  similarity: 0.002   False Positive
# Suspicious file:  g0pE_taskd.txt  similarity: 0.001   False Positive
# -----------------------------------------------------------
# Total Number of False Positives:  104


# False Negatives: 0
# ---------------------------------------------------------------------
# Original file:  orig_taska.txt
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# Original file:  orig_taskb.txt
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# Original file:  orig_taskc.txt
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# Original file:  orig_taskd.txt
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# Original file:  orig_taske.txt
# ---------------------------------------------------------------------
# Total Number of False Negatives:  0